# DIY Interactive Segmentation with napari

napari is a very flexible and "hackable" tool. In this tutorial we will
make a custom interactive segmentation tool from scratch.

In this tutorial we will write an interactive segmentation tool and use it on data hosted on [Zebrahub](https://zebrahub.ds.czbiohub.org/).

![3D volumetric image from zebrahub of a developing zebrafish](resources/zebrahub_volume.png)
![3D volumetric segmentation of with some of the zebrahub image segmented but not very well](resources/zebrahub_partial_segmentation.png)

!pip install scikit-learn scikit-image ome-zarr

## Setup

In [68]:
from appdirs import user_data_dir
import os
import zarr
import dask.array as da
import toolz as tz

from sklearn.ensemble import RandomForestClassifier

from skimage import data, segmentation, feature, future
from skimage.feature import multiscale_basic_features
from skimage.io import imread, imshow
import numpy as np
from functools import partial
import napari
import threading

from ome_zarr.io import parse_url
from ome_zarr.reader import Reader

from functools import partial
from psygnal import debounced
from superqt import ensure_main_thread

import logging
import sys

LOGGER = logging.getLogger("halfway_to_i2k_2023_america")
LOGGER.setLevel(logging.DEBUG)

streamHandler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter(
    "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
streamHandler.setFormatter(formatter)
LOGGER.addHandler(streamHandler)

## Reading in data

Get data from OpenOrganelle.

In [69]:
parent_path =r'D:\images\tnia-python-images\imagesc\2024_12_18_noisy_cellpose_3D'
im = imread(os.path.join(parent_path, r'Sample_data.tif'))
print(im.shape)

(7, 1024, 1024)


### Visualize in napari

In [70]:
viewer = napari.Viewer()

scale = (3, 1, 1)
contrast_limits = (0,im.max())

data_layer = viewer.add_image(im, scale=scale, contrast_limits=contrast_limits)
data_layer.bounding_box.visible = True

### Extracting features

In [71]:
def extract_features(image, feature_params):
    features_func = partial(
        multiscale_basic_features,
        intensity=feature_params["intensity"],
        edges=feature_params["edges"],
        texture=feature_params["texture"],
        sigma_min=feature_params["sigma_min"],
        sigma_max=feature_params["sigma_max"],
        channel_axis=None,
    )
    # print(f"image shape {image.shape} feature params {feature_params}")
    features = features_func(np.squeeze(image))
    return features

example_feature_params = {
    "sigma_min": 1,
    "sigma_max": 5,
    "intensity": True,
    "edges": True,
    "texture": True,
}

features = extract_features(im, example_feature_params)
features.shape

(7, 1024, 1024, 15)

### Visualize Features

What do these features we are extracting look like?

![A set of features used for the zebrahub model in this tutorial](resources/zebrahub_features.png)

In [72]:
def show_features():
    for feature_idx in range(features.shape[-1]):
        viewer.add_image(features[:, :, :, feature_idx])
        
#show_features()

### Making the Interactive Segmentation Tool!

Ok, now we've seen:
- our data
- some features we can compute for our data

Our goal is to create an image where we have labels that correspond to the zebrafish sample. 

The approach is that when we annotate/draw in our painting layer, then we want our segmentations to be updated automatically. 

We will do this using 3 different image layers:

1. Our input image
2. A layer for painting
3. A layer for storing the machine learning generated predictions

Due to popular demand we will be using Zarr to store these layers, because that will help this approach scale to very large datasets. However, we could have used numpy arrays as well.

### Create our painting and prediction layers

In [73]:
im.shape

(7, 1024, 1024)

In [74]:
zarr_path = './'
print(f"Saving outputs to zarr path: {zarr_path}")

# Create a prediction layer
prediction_data = zarr.open(
    f"{zarr_path}/prediction",
    mode='a',
    shape=im.shape,
    dtype='i4',
    dimension_separator="/",

)
prediction_layer = viewer.add_labels(prediction_data, name="Prediction", scale=data_layer.scale)

# Create a painting layer
painting_data = zarr.open(
    f"{zarr_path}/painting",
    mode='a',
    shape=im.shape,
    dtype='i4',
    dimension_separator="/",
)
painting_layer = viewer.add_labels(painting_data, name="Painting", scale=data_layer.scale)

Saving outputs to zarr path: ./


In [75]:
painting_data.shape

(7, 1024, 1024)

### Let's make a UI as well

![A UI widget showing controls for feature size and type](resources/diy_interactive_segmentation_widget.png)

In [76]:
from qtpy.QtWidgets import (
    QVBoxLayout,
    QHBoxLayout,
    QComboBox,
    QLabel,
    QCheckBox,
    QDoubleSpinBox,
    QGroupBox,
    QWidget,
)

class NapariMLWidget(QWidget):
    def __init__(self, parent=None):
        super(NapariMLWidget, self).__init__(parent)

        self.initUI()

    def initUI(self):
        layout = QVBoxLayout()

        # Dropdown for selecting the model
        model_label = QLabel("Select Model")
        self.model_dropdown = QComboBox()
        self.model_dropdown.addItems(["Random Forest"])
        model_layout = QHBoxLayout()
        model_layout.addWidget(model_label)
        model_layout.addWidget(self.model_dropdown)
        layout.addLayout(model_layout)

        # Select the range of sigma sizes
        self.sigma_start_spinbox = QDoubleSpinBox()
        self.sigma_start_spinbox.setRange(0, 256)
        self.sigma_start_spinbox.setValue(1)

        self.sigma_end_spinbox = QDoubleSpinBox()
        self.sigma_end_spinbox.setRange(0, 256)
        self.sigma_end_spinbox.setValue(5)

        sigma_layout = QHBoxLayout()
        sigma_layout.addWidget(QLabel("Sigma Range: From"))
        sigma_layout.addWidget(self.sigma_start_spinbox)
        sigma_layout.addWidget(QLabel("To"))
        sigma_layout.addWidget(self.sigma_end_spinbox)
        layout.addLayout(sigma_layout)

        # Boolean options for features
        self.intensity_checkbox = QCheckBox("Intensity")
        self.intensity_checkbox.setChecked(True)
        self.edges_checkbox = QCheckBox("Edges")
        self.texture_checkbox = QCheckBox("Texture")
        self.texture_checkbox.setChecked(True)

        features_group = QGroupBox("Features")
        features_layout = QVBoxLayout()
        features_layout.addWidget(self.intensity_checkbox)
        features_layout.addWidget(self.edges_checkbox)
        features_layout.addWidget(self.texture_checkbox)
        features_group.setLayout(features_layout)
        layout.addWidget(features_group)

        # Dropdown for data selection
        data_label = QLabel("Select Data for Model Fitting")
        self.data_dropdown = QComboBox()
        self.data_dropdown.addItems(
            ["Current Displayed Region", "Whole Image"]
        )
        self.data_dropdown.setCurrentText("Current Displayed Region")
        data_layout = QHBoxLayout()
        data_layout.addWidget(data_label)
        data_layout.addWidget(self.data_dropdown)
        layout.addLayout(data_layout)

        # Checkbox for live model fitting
        self.live_fit_checkbox = QCheckBox("Live Model Fitting")
        self.live_fit_checkbox.setChecked(True)
        layout.addWidget(self.live_fit_checkbox)

        # Checkbox for live prediction
        self.live_pred_checkbox = QCheckBox("Live Prediction")
        self.live_pred_checkbox.setChecked(True)
        layout.addWidget(self.live_pred_checkbox)

        self.setLayout(layout)
        

In [77]:
# Let's add this widget to napari

widget = NapariMLWidget()
viewer.window.add_dock_widget(widget, name="halfway to I2K 2023 America")

### We have a widget, we have our painting and prediction layers, now what?

We need to start connecting things together. How should we do that? napari has things called "events" that happen when things happen within napari. We want to respond to a few different event types:
- changes in camera (e.g. camera position and rotation)
- changes in "dims" (e.g. moving a dimension slider)
- painting events (e.g. a user clicked, painted, and release their mouse)

When one of these events happens, we want to: 
- update our machine learning model with the new painted data
- update our prediction with the updated ML model

In [78]:
# Let's start with our event listener

# We use "curry" because this allows us to "store" our viewer and widget for later use
@tz.curry
def on_data_change(event, viewer=None, widget=None):
    corner_pixels = data_layer.corner_pixels

    # Ensure the painting layer visual is updated
    painting_layer.refresh()

    # Training the ML model and generating predictions can take time
    #   we will use a "thread" to perform these calculations
    #   otherwise napari will freeze until these
    # calculations are done
    thread = threading.Thread(
        target=threaded_on_data_change,
        args=(
            event,
            corner_pixels,
            viewer.dims,
            widget.model_dropdown.currentText(),
            {
                "sigma_min": widget.sigma_start_spinbox.value(),
                "sigma_max": widget.sigma_end_spinbox.value(),
                "intensity": widget.intensity_checkbox.isChecked(),
                "edges": widget.edges_checkbox.isChecked(),
                "texture": widget.texture_checkbox.isChecked(),
            },
            widget.live_fit_checkbox.isChecked(),
            widget.live_pred_checkbox.isChecked(),
            widget.data_dropdown.currentText(),
        ),
    )
    thread.start()
    thread.join()

    # Ensure the prediction layer visual is updated
    prediction_layer.refresh()

In [ ]:
# Now we have to make the hard part of the listener

model = None

def threaded_on_data_change(
    event,
    corner_pixels,
    dims,
    model_type,
    feature_params,
    live_fit,
    live_prediction,
    data_choice,
):
    global model
    LOGGER.info(f"Labels data has changed! {event}")

    current_step = dims.current_step

    # Find a mask of indices we will use for fetching our data
    mask_idx = (slice(viewer.dims.current_step[0], viewer.dims.current_step[0]+1), slice(corner_pixels[0, 1], corner_pixels[1, 1]), slice(corner_pixels[0, 2], corner_pixels[1, 2]))
    if data_choice == "Whole Image":
        mask_idx = tuple([slice(0, sz) for sz in data_layer.data.shape])

    LOGGER.info(f"mask idx {mask_idx}, image {data_layer.data.shape}")
    active_image = data_layer.data[mask_idx]
    LOGGER.info(
        f"active image shape {active_image.shape} data choice {data_choice} painting_data {painting_data.shape} mask_idx {mask_idx}"
    )

    active_labels = painting_data[mask_idx]

    def compute_features(image, feature_params):
        """Compute features for each channel and concatenate them."""
        features = extract_features(
            image, feature_params
        )

        return features

    training_labels = None

    #if data_choice == "Current Displayed Region":
    # Use only the currently displayed region.
    training_features = compute_features(
        active_image, feature_params
    )
    training_labels = np.squeeze(active_labels)

    #else:
    #    raise ValueError(f"Invalid data choice: {data_choice}")

    if (training_labels is None) or np.any(training_labels.shape == 0):
        LOGGER.info("No training data yet. Skipping model update")
    elif live_fit:
        # Retrain model
        LOGGER.info(
            f"training model with labels {training_labels.shape} features {training_features.shape} unique labels {np.unique(training_labels[:])}"
        )
        model = update_model(training_labels, training_features, model_type)

    # Don't do live prediction on whole image, that happens earlier slicewise
    if live_prediction:
        # Update prediction_data
        prediction_features = compute_features(
            active_image, feature_params
        )
        # Add 1 becasue of the background label adjustment for the model
        prediction = predict(model, prediction_features, model_type)
        LOGGER.info(
            f"prediction {prediction.shape} prediction layer {prediction_layer.data.shape} prediction {np.transpose(prediction).shape} features {prediction_features.shape}"
        )

        if data_choice == "Whole Image":
            prediction_layer.data[mask_idx] = np.transpose(prediction)
        else:
            prediction_layer.data[mask_idx] = np.transpose(prediction)[
                np.newaxis, :
            ]

2024-12-18 07:35:31,474 - halfway_to_i2k_2023_america - INFO - Labels data has changed! Event
2024-12-18 07:35:31,474 - halfway_to_i2k_2023_america - INFO - Labels data has changed! Event
2024-12-18 07:35:31,474 - halfway_to_i2k_2023_america - INFO - Labels data has changed! Event
2024-12-18 07:35:31,474 - halfway_to_i2k_2023_america - INFO - Labels data has changed! Event
2024-12-18 07:35:31,474 - halfway_to_i2k_2023_america - INFO - Labels data has changed! Event
2024-12-18 07:35:31,474 - halfway_to_i2k_2023_america - INFO - Labels data has changed! Event
2024-12-18 07:35:31,474 - halfway_to_i2k_2023_america - INFO - Labels data has changed! Event
2024-12-18 07:35:31,488 - halfway_to_i2k_2023_america - INFO - mask idx (slice(0, 7, None), slice(0, 1024, None), slice(0, 1024, None)), image (7, 1024, 1024)
2024-12-18 07:35:31,488 - halfway_to_i2k_2023_america - INFO - mask idx (slice(0, 7, None), slice(0, 1024, None), slice(0, 1024, None)), image (7, 1024, 1024)
2024-12-18 07:35:31,488 

In [80]:
# Model training function that respects widget's model choice
def update_model(labels, features, model_type):
    features = features[labels > 0, :]
    # We shift labels - 1 because background is 0 and has special meaning, but models need to start at 0
    labels = labels[labels > 0] - 1
    
    if model_type == "Random Forest":
        clf = RandomForestClassifier(
            n_estimators=50, n_jobs=-1, max_depth=10, max_samples=0.05
        )

    print(
        f"updating model with label shape  {labels.shape} feature shape {features.shape} unique labels {np.unique(labels)}"
    )
    
    clf.fit(features, labels)

    return clf


def predict(model, features, model_type):
    # We shift labels + 1 because background is 0 and has special meaning
    prediction = future.predict_segmenter(features.reshape(-1, features.shape[-1]), model).reshape(features.shape[:-1]) + 1

    return np.transpose(prediction)

In [ ]:
# Now connect everything together
for listener in [
    viewer.camera.events,
    viewer.dims.events,
    painting_layer.events.paint,
]:
    listener.connect(
        debounced(
            ensure_main_thread(
                on_data_change(
                    viewer=viewer,
                    widget=widget,  # pass the widget instance for easy access to settings
                )
            ),
            timeout=1000,
        )
    )

2024-12-18 07:32:02,484 - halfway_to_i2k_2023_america - INFO - Labels data has changed! Event
2024-12-18 07:32:02,484 - halfway_to_i2k_2023_america - INFO - Labels data has changed! Event
2024-12-18 07:32:02,484 - halfway_to_i2k_2023_america - INFO - Labels data has changed! Event
2024-12-18 07:32:02,484 - halfway_to_i2k_2023_america - INFO - Labels data has changed! Event
2024-12-18 07:32:02,484 - halfway_to_i2k_2023_america - INFO - Labels data has changed! Event
2024-12-18 07:32:02,484 - halfway_to_i2k_2023_america - INFO - Labels data has changed! Event
2024-12-18 07:32:02,484 - halfway_to_i2k_2023_america - INFO - Labels data has changed! Event
2024-12-18 07:32:02,490 - halfway_to_i2k_2023_america - INFO - mask idx (slice(0, 7, None), slice(0, 1024, None), slice(0, 1024, None)), image (7, 1024, 1024)
2024-12-18 07:32:02,490 - halfway_to_i2k_2023_america - INFO - mask idx (slice(0, 7, None), slice(0, 1024, None), slice(0, 1024, None)), image (7, 1024, 1024)
2024-12-18 07:32:02,490 

Exception in thread Thread-4122 (threaded_on_data_change):
Traceback (most recent call last):
  File "c:\Users\bnort\miniconda3\envs\pytorch_and_SAM3\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\bnort\miniconda3\envs\pytorch_and_SAM3\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\bnort\miniconda3\envs\pytorch_and_SAM3\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\bnort\AppData\Local\Temp\ipykernel_33040\3381255996.py", line 69, in threaded_on_data_change
  File "C:\Users\bnort\AppData\Local\Temp\ipykernel_33040\1753605063.py", line 23, in predict
  File "c:\Users\bnort\miniconda3\envs\pytorch_and_SAM3\lib\site-packages\skimage\future\trainable_segmentation.py", line 148, in predict_segmenter
    predicted_labels = clf.predict(features)
AttributeError: 'NoneType' object has no attribute 'predict'


2024-12-18 07:32:14,756 - halfway_to_i2k_2023_america - INFO - Labels data has changed! Event
2024-12-18 07:32:14,756 - halfway_to_i2k_2023_america - INFO - Labels data has changed! Event
2024-12-18 07:32:14,756 - halfway_to_i2k_2023_america - INFO - Labels data has changed! Event
2024-12-18 07:32:14,756 - halfway_to_i2k_2023_america - INFO - Labels data has changed! Event
2024-12-18 07:32:14,756 - halfway_to_i2k_2023_america - INFO - Labels data has changed! Event
2024-12-18 07:32:14,756 - halfway_to_i2k_2023_america - INFO - Labels data has changed! Event
2024-12-18 07:32:14,756 - halfway_to_i2k_2023_america - INFO - Labels data has changed! Event
2024-12-18 07:32:14,763 - halfway_to_i2k_2023_america - INFO - mask idx (slice(0, 7, None), slice(0, 1024, None), slice(0, 1024, None)), image (7, 1024, 1024)
2024-12-18 07:32:14,763 - halfway_to_i2k_2023_america - INFO - mask idx (slice(0, 7, None), slice(0, 1024, None), slice(0, 1024, None)), image (7, 1024, 1024)
2024-12-18 07:32:14,763 

Exception in thread Thread-4125 (threaded_on_data_change):
Traceback (most recent call last):
  File "c:\Users\bnort\miniconda3\envs\pytorch_and_SAM3\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\bnort\miniconda3\envs\pytorch_and_SAM3\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\bnort\miniconda3\envs\pytorch_and_SAM3\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\bnort\AppData\Local\Temp\ipykernel_33040\3381255996.py", line 69, in threaded_on_data_change
  File "C:\Users\bnort\AppData\Local\Temp\ipykernel_33040\1753605063.py", line 23, in predict
  File "c:\Users\bnort\miniconda3\envs\pytorch_and_SAM3\lib\site-packages\skimage\future\trainable_segmentation.py", line 148, in predict_segmenter
    predicted_labels = clf.predict(features)
AttributeError: 'NoneType' object has no attribute 'predict'
